Noise Suppression and Normalization and Standardization

In [ ]:
%pip install opencv-python numpy tqdm scikit-image matplotlib

Application of Anisotropic Diffusion filtering and CLAHE[Contrast Limited Adaptive Histogram Equalization]

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# === Anisotropic Diffusion Function ===
def anisotropic_diffusion(img, niter=10, kappa=50, gamma=0.1, option=1):
    img = img.astype('float32')
    for i in range(niter):
        nablaN = np.roll(img, -1, axis=0) - img
        nablaS = np.roll(img, 1, axis=0) - img
        nablaE = np.roll(img, -1, axis=1) - img
        nablaW = np.roll(img, 1, axis=1) - img

        if option == 1:
            cN = np.exp(-(nablaN / kappa) ** 2)
            cS = np.exp(-(nablaS / kappa) ** 2)
            cE = np.exp(-(nablaE / kappa) ** 2)
            cW = np.exp(-(nablaW / kappa) ** 2)
        elif option == 2:
            cN = 1.0 / (1.0 + (nablaN / kappa) ** 2)
            cS = 1.0 / (1.0 + (nablaS / kappa) ** 2)
            cE = 1.0 / (1.0 + (nablaE / kappa) ** 2)
            cW = 1.0 / (1.0 + (nablaW / kappa) ** 2)

        img += gamma * (cN * nablaN + cS * nablaS + cE * nablaE + cW * nablaW)
    return img

# === Paths ===
# Define the base path and result path

base_path = 'brain_tumor_dataset'
result_path = os.path.join(base_path, 'result')

# Ensure result subfolders exist
for subfolder in ['yes', 'no']:
    os.makedirs(os.path.join(result_path, subfolder), exist_ok=True)

# Process both 'yes' and 'no' folders
for label in ['yes', 'no']:
    folder_path = os.path.join(base_path, label)
    save_path = os.path.join(result_path, label)
    # Ensure the save path exists
    for filename in tqdm(os.listdir(folder_path), desc=f'Processing {label}'):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Apply filtering
            diffused = anisotropic_diffusion(img, niter=15, kappa=30, gamma=0.2) # Anisotropic diffusion filtering
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)) # Contrast Limited Adaptive Histogram Equalization
            enhanced = clahe.apply(diffused.astype('uint8')) # Apply CLAHE

            # Save to category-specific folder
            name, ext = os.path.splitext(filename)
            cv2.imwrite(os.path.join(save_path, f"{label}_{name}_org.png"), img)
            cv2.imwrite(os.path.join(save_path, f"{label}_{name}_op.png"), enhanced)


DIfference between original image and output image and errors

In [ ]:
from skimage import io, img_as_float
from skimage.transform import resize
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio, structural_similarity
import matplotlib.pyplot as plt
import numpy as np
import os

# === Paths ===
base_result = 'brain_tumor_dataset/result'
heatmap_base = os.path.join(base_result, 'heatmaps')
output_txt = os.path.join(base_result, 'results.txt')

# Ensure heatmap subfolders exist
for category in ['yes', 'no']:
    os.makedirs(os.path.join(heatmap_base, category), exist_ok=True)

# Clear or create the results file
with open(output_txt, 'w') as f:
    f.write("Filename\t\tMSE\t\tPSNR(dB)\tSSIM\t\tAvg_%_Mod\n")

# Process images from result/yes and result/no
for category in ['yes', 'no']:
    category_path = os.path.join(base_result, category)

    for file in os.listdir(category_path):
        if file.endswith('_org.png'):
            base_name = file.replace('_org.png', '')
            org_path = os.path.join(category_path, file)
            op_path = os.path.join(category_path, f"{base_name}_op.png")

            if not os.path.exists(op_path):
                continue

            # Load images
            img_original = io.imread(org_path, as_gray=True)
            img_denoised = io.imread(op_path, as_gray=True)

            # Resize if needed
            if img_original.shape != img_denoised.shape:
                img_denoised = resize(img_denoised, img_original.shape, anti_aliasing=True)

            # Convert to float
            img_original = img_as_float(img_original)
            img_denoised = img_as_float(img_denoised)

            # Compute metrics
            mse = mean_squared_error(img_original, img_denoised)
            psnr = peak_signal_noise_ratio(img_original, img_denoised)
            ssim = structural_similarity(img_original, img_denoised, data_range=1.0)
            mad = np.mean(np.abs(img_original - img_denoised))
            percent_modified = mad * 100

            # Write to text file
            with open(output_txt, 'a') as f:
                f.write(f"{category}_{base_name}\t{mse:.4f}\t{psnr:.2f}\t\t{ssim:.4f}\t\t{percent_modified:.2f}\n")

            # Save heatmap
            diff = np.abs(img_original - img_denoised)
            plt.imshow(diff, cmap='hot')
            plt.axis('off')
            plt.tight_layout(pad=0)

            heatmap_path = os.path.join(heatmap_base, category, f"{base_name}_heatmap.png")
            plt.savefig(heatmap_path, bbox_inches='tight', pad_inches=0)
            plt.close()


Normalization and Standardization

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# === Paths ===
base_path = 'brain_tumor_dataset/result'
output_path = 'brain_tumor_dataset/normalized and standardized'

# Create output dirs
for label in ['yes', 'no']:
    os.makedirs(os.path.join(output_path, label), exist_ok=True)

# === Process images ===
for label in ['yes', 'no']:
    folder = os.path.join(base_path, label)
    out_folder = os.path.join(output_path, label)

    for file in tqdm(os.listdir(folder), desc=f"Processing {label}"):
        if file.endswith('_op.png'):
            img_path = os.path.join(folder, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Normalize to [0, 1]
            img_norm = img.astype(np.float32) / 255.0

            # Standardize to mean=0, std=1
            mean = np.mean(img_norm)
            std = np.std(img_norm)
            if std == 0: std = 1e-6  # Avoid division by zero
            img_std = (img_norm - mean) / std

            # Rescale back to 0–255 for saving (optional for visualization)
            img_out = cv2.normalize(img_std, None, 0, 255, cv2.NORM_MINMAX)
            img_out = img_out.astype(np.uint8)

            # Save
            name, _ = os.path.splitext(file)
            save_path = os.path.join(out_folder, f"{name}_std.png")
            cv2.imwrite(save_path, img_out)
